In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_csv('추자도.csv', encoding='cp949')

df = df.rename(columns={'일시': 'date', '평균기압(hPa)': 'Apress(hPa)', '평균 기온(°C)': 'Atemp(°C)', '평균 수온(°C)': 'Stemp(°C)', '평균 상대습도(%)': 'Moisture(%)'})
df=df.set_index(pd.to_datetime(df.date)) # Date를 인덱스로
df=df[['Apress(hPa)','Atemp(°C)','Stemp(°C)','Moisture(%)']] # Column 재정리
df.dropna(inplace=True)
df

,Apress(hPa),Atemp(°C),Stemp(°C),Moisture(%)
date,,,,
2014-01-16,1029.6,4.4,14.6,60.0
2014-01-17,1026.9,6.1,14.8,59.0
2014-01-18,1028.5,5.8,14.9,66.0
2014-01-19,1029.1,4.6,14.8,66.0
2014-01-20,1022.0,7.6,14.6,64.0
...,...,...,...,...
2022-12-27,1028.8,5.6,16.1,63.0
2022-12-28,1030.4,5.6,16.0,59.0
2022-12-29,1032.6,4.9,16.1,59.0


In [3]:
from sklearn.linear_model import LinearRegression
# Create a linear regression model
regression_model = LinearRegression()

# Prepare the data for regression
X = df.index.to_julian_date().values.reshape(-1, 1)  # Convert date index to Julian dates
y = df['Stemp(°C)'].values.reshape(-1, 1)

# Fit the linear regression model
regression_model.fit(X, y)

# Predict the next 7 days
next_dates = pd.date_range(start=df.index[-1] + pd.DateOffset(days=1), periods=7)
next_dates_julian = next_dates.to_julian_date().values.reshape(-1, 1)
next_temperatures = regression_model.predict(next_dates_julian)

# Create a DataFrame to store the predictions
predictions = pd.DataFrame({'Date': next_dates, 'Predicted Temperature': next_temperatures.flatten()})
predictions.set_index('Date', inplace=True)

print(predictions)

            Predicted Temperature
Date                             
2023-01-01              18.799539
2023-01-02              18.799766
2023-01-03              18.799992
2023-01-04              18.800219
2023-01-05              18.800445
2023-01-06              18.800672
2023-01-07              18.800899


In [4]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Extend the DataFrame to include the future date
future_date = pd.to_datetime('2023-01-01')
df.loc[future_date] = None

# Fill any missing values in the DataFrame (if necessary)
df.fillna(method='ffill', inplace=True)

# Extract the relevant series for water temperature
stemp = df['Stemp(°C)']

# Fit the ARIMA model
model = ARIMA(stemp, order=(1, 1, 1))
model_fit = model.fit()

# Forecast the water temperature for the extended date range
forecast = model_fit.forecast(steps=len(df))

# Get the forecasted value for 2023-01-01
forecast_value = forecast.iloc[-1]  # Retrieve the last forecasted value

# Print the forecasted water temperature for 2023-01-01
print("Forecasted water temperature for 2023-01-01:", forecast_value)

C:\Users\admin\AppData\Local\Temp\ipykernel_14072\1148420423.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[future_date] = None
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dat

Forecasted water temperature for 2023-01-01: 16.19974655936624


c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [5]:
# Extend the DataFrame to include the future dates
future_dates = pd.date_range(start='2023-01-01', end='2023-01-03')
df = df.append(pd.DataFrame(index=future_dates))

# Fill any missing values in the DataFrame (if necessary)
df.fillna(method='ffill', inplace=True)

# Extract the relevant series for water temperature
stemp = df['Stemp(°C)']

# Fit the ARIMA model
model = ARIMA(stemp, order=(1, 1, 1))
model_fit = model.fit()

# Forecast the water temperature for the future dates
forecast_values = model_fit.forecast(steps=len(future_dates))

# Print the forecasted values and dates
for date, value in zip(future_dates, forecast_values):
    print("Forecasted water temperature for", date.date(), ":", value)

C:\Users\admin\AppData\Local\Temp\ipykernel_14072\2006745364.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(index=future_dates))
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dat

Forecasted water temperature for 2023-01-01 : 16.200076941287854
Forecasted water temperature for 2023-01-02 : 16.20004032094031
Forecasted water temperature for 2023-01-03 : 16.200057750462115


c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [6]:
# Extend the DataFrame to include the future dates
future_dates = pd.date_range(start='2023-01-01', end='2023-01-03')
df = df.append(pd.DataFrame(index=future_dates))

# Fill any missing values in the DataFrame (if necessary)
df.fillna(method='ffill', inplace=True)

# Extract the relevant series for water temperature
stemp = df['Stemp(°C)']

# Fit the ARIMA model
model = ARIMA(stemp, order=(1, 1, 1))
model_fit = model.fit()

# Forecast the water temperature for the future dates
forecast_values = model_fit.forecast(steps=len(future_dates))

# Print the forecasted values and dates
for date, value in zip(future_dates, forecast_values):
    print("Forecasted water temperature for", date.date(), ":", value)

C:\Users\admin\AppData\Local\Temp\ipykernel_14072\2006745364.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(index=future_dates))
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admi

Forecasted water temperature for 2023-01-01 : 16.199984173949442
Forecasted water temperature for 2023-01-02 : 16.19999170642639
Forecasted water temperature for 2023-01-03 : 16.199988121311534


c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [7]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Extend the DataFrame to include the future dates
future_dates = pd.date_range(start='2023-01-01', end='2023-01-02')
df = df.append(pd.DataFrame(index=future_dates))

# Fill any missing values in the DataFrame (if necessary)
df.fillna(method='ffill', inplace=True)

# Extract the relevant series for water temperature
stemp = df['Stemp(°C)']

# Extract the relevant series for the exogenous variable
atemp = df['Atemp(°C)']

# Fit the SARIMA model with exogenous variable
model = SARIMAX(stemp, exog=atemp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
model_fit = model.fit()

# Forecast the water temperature for the future dates with exogenous variable
forecast_values = model_fit.get_forecast(steps=len(future_dates), exog=atemp[len(df)-len(future_dates):])

# Print the forecasted values and dates
for date, value in zip(future_dates, forecast_values.predicted_mean):
    print("Forecasted water temperature for", date.date(), ":", value)

C:\Users\admin\AppData\Local\Temp\ipykernel_14072\161391055.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(index=future_dates))
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\admin

Forecasted water temperature for 2023-01-01 : 16.26961494334818
Forecasted water temperature for 2023-01-02 : 16.261745291685934


c:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
